In [ ]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
#model and 
from utils import *
X,labels=read_data('datasets/data sets/MSRC-v1.mat')
X = [X[0,i].T for  i  in range(X.shape[1])]
labels  = labels.squeeze()
from models import MSE,MDcR,DSE
model_MSE = MSE()
model_MDcR = MDcR()
model_DSE = DSE()

res_nim = []
for d_ in range(10,12):
    Y = model_MSE.mse(X,gamma = 0.9,d_=d_,t=1e8,epoch=500)
    Y = model_MDcR.mdcr(X,d_=10,t=1e8,lmd=10,epoch=500)
    nmi_list = []
    for i in range(20):
        pred = kmeans(Y,7)
        # criterion
        from sklearn.metrics.cluster import normalized_mutual_info_score
        nmi = normalized_mutual_info_score(labels,pred)
        nmi_list.append(nmi)
    nmi = np.mean(nmi_list)
    res_nim.append(nmi)


In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(10,12),res_nim)

In [42]:
#DSE 單獨
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

from utils import *
X,labels=read_data('datasets/data sets/MSRC-v1.mat')
X = [X[0,i].T for  i  in range(X.shape[1])]
labels  = labels.squeeze()

from models import DSE
model_DSE = DSE()
pred = model_DSE.dse(X,5,5,500)
from sklearn.metrics.cluster import normalized_mutual_info_score
nmi = normalized_mutual_info_score(labels,pred)
print(nmi)
pred

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0.3647486277878456


array([3, 1, 3, 3, 4, 0, 0, 4, 1, 1, 3, 3, 0, 0, 0, 1, 0, 3, 3, 0, 0, 0,
       3, 0, 0, 0, 0, 0, 0, 0, 4, 3, 4, 3, 4, 3, 1, 1, 0, 3, 1, 1, 1, 4,
       1, 3, 0, 1, 1, 4, 0, 4, 4, 4, 4, 0, 0, 0, 4, 0, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 0, 4, 4, 4, 4, 4, 4,
       4, 4, 1, 1, 2, 2, 1, 2, 2, 1, 1, 3, 2, 1, 2, 2, 2, 2, 2, 1, 3, 2,
       3, 1, 3, 1, 3, 3, 0, 2, 3, 4, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 3, 3,
       1, 3, 1, 3, 1, 1, 0, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4,
       4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 4, 4, 1, 1, 1, 1,
       1, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 3], dtype=int64)

In [18]:
X[0].shape

(24, 210)

In [ ]:
x=np.random.rand(3,4)
np.exp(x)/np.exp(x).sum(1).reshape(3,-1)